# Project 2

    The purpose of this assignment is to give you a chance to get some hands-on experience implementing anonparametric algorithm to perform classification and regression. Specifically, you will be implementing ak-nearest neighbor classifier and regressor. Be careful with how the attributes are handled. Nearest neighbormethods work best with numeric attributes, so some care will need to be taken to handle categorical (i.e.,discrete) attributes.
    
    In this project, and all future projects, the experimental design we will use is called 5-fold cross-validation.The basic idea is that you are going to take your data set and divide it into five equally-sized subsets. Whenyou do this, you should select the data points randomly, but with a twist. Ultimately, you would like thesame number of examples to be in each class in each of the five partitions. This is called “stratified” cross-validation. For example, if you have a data set of 100 points where 1/3 of the data is in one class and 2/3of the data is in another class, you will create five partitions of 20 examples each. Then for each of thesepartitions, 1/3 of the examples (around 6 or 7 points) should be from the one class, and the remaining pointsshould be in the other class.
    
    Note that stratification is not expected for the regression data sets.  You should be sure to sampleuniformly across all of the response values (i.e., targets) when creating your folds. One approach for doingthat (that’s not particularly random) is to sort the data on the response variable and take every fifth pointfor a given fold.
    
    With five-fold cross-validation, you will run five experiments where you train on four of the partitions(so 80% of the data) and test on the remaining partition (20% of the data). You will rotate through thepartitions so that each one serves as a test set exactly once. Then you will average the performance on thesefive test-set partitions when you report the results.
    
    Let’s talk again about tuning.  As before, extract 10% of the data to be used for tuning.  For yourtraining set, test against this 10% with different parameter values and pick the best model. Then apply themodel that goes with those tuned values against your test set. To be specific, since you are doing 5-foldcross-validation, you first take out 10% for tuning. Then from the remaining 90% split into five folds of 18%of the data each. For each of the five experiments, combine four of the folds, holding out the fifth as yourtest set. Train on the four folds while tuning with the 10%. Take the result and evaluate generalizationability on the held-out fold. Repeat this process five times for each of the folds but using the same 10% fortuning.
    

For this project, the following steps are required:

1) Download the six (6) data sets from the UCI Machine Learning repository. You can find this repositoryathttp://archive.ics.uci.edu/ml/. Again, all data sets are also available in Blackboard. All ofthe specific URLs are also provided above.

2) Implementk-nearest neighbor and be prepared to find the bestkvalue for your experiments. Youmust tunekand explain in your report how you did the tuning.

3) Implement editedk-nearest neighbor. See above with respect to tuningk. On the regression problems,you should define an error thresholdto determine if a prediction is correct or not. Thiswill needto be tuned.

4) Implement condensedk-nearest neighbor. See above with respect to tuningkand.

5) For classification, employ a plurality vote to determine the class. For regression, apply a Gaussian(radial basis function) kernel to make your prediction. You will need to tune the bandwidthσfor theGaussian kernel.

6) Run your algorithms on each of the data sets. These runs should be done with 5-fold cross-validationso you can compare your results statistically. You can use classification error or mean squared error(as appropriate) for your loss function.

7) Write a very brief paper that incorporates the following elements, summarizing the results of yourexperiments. Your paper is required to be at least 5 pages and no more than 10 pages using the JMLRformat You can find templates for this format athttp://www.jmlr.org/format/format.html. Theformat is also available within Overleaf.1. Title and author name2. Problem statement, including hypothesis, projecting how you expect each algorithm to perform3. Brief description of your experimental approach, including any assumptions made with your al-gorithms4. Presentation of the results of your experiments5. A discussion of the behavior of your algorithms, combined with any conclusions you can draw6. Summary7. References (Only required if you use a resource other than the course content.)

## Data Import

In [1]:
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

# Imports 
import numpy as np
import scipy.stats as stats
# import seaborn as sns
# import matplotlib.pyplot as plt
import pandas as pd
import random
import math
from random import seed
from random import randrange
import time
import timeit


In [2]:
#Load Data
#Abalone Data
Abalone = pd.read_csv("abalone.data", sep = ',')
Abalone.columns = ["Sex", "Length","Diameter", "Height","WholeWeight","ShuckedWeight","VisceraWeight","ShellWeight", "Rings"]

# ForestFire Data
ForestFire = pd.read_csv("forestfires.data", sep = ',')
ForestFire.columns = ["X_Cord", "Y_Cord","Month", "Day","FFMC","DMC","DC","ISI", "Temp", "RH", "Wind", "Rain", "Area"]

Glass = pd.read_csv("glass.data", sep = ',')
Glass.columns = ["ID", "RefractiveIndex","Sodium", "Mag","Alum","Silicon","K","Calcium", "Barium", "Iron", "GlassType"]

HouseVote = pd.read_csv("house-votes-84.data", sep = ',')
HouseVote.columns = ["Class", "Handicap","Water", "Adoption","Physician", "SalvadorAid", "ReligionInSchool", "SatelitteTestBan", "NicaraguanAid","Missile", "Immigration","CorpCutBack","Education","Sue","Crime","DutyFreeExport","ExportAdmin"]
HouseVote = HouseVote[["Handicap","Water", "Adoption","Physician", "SalvadorAid", "ReligionInSchool", "SatelitteTestBan", "NicaraguanAid","Missile", "Immigration","CorpCutBack","Education","Sue","Crime","DutyFreeExport","ExportAdmin", "Class"]]

Machine = pd.read_csv("machine.data", sep = ',')
Machine.columns = ["Vendor", "Model","MYCT", "MMIN","MMAX", "CACH", "CHMIN", "CHMAX", "PRP","ERP"]
Machine = Machine[["Model","MYCT", "MMIN","MMAX", "CACH", "CHMIN", "CHMAX", "PRP","ERP","Vendor"]]

Segmentation = pd.read_csv("segmentation.data")
#Segmentation.columns = ["Region_COL", "Region_ROW","SLD5", "SLD2","VedgeMean", "VedgeSd", "HedgeMean", "HedgeSd", "IntensityMean","rawredM", "rawblueM", "rawgreenM","exredM", "exblueM", "exgreenM", "ValueM", "SatM", "HueM"]
Segmentation.to_csv('Segmentation.csv', header=False, index=False)
SegmentationC = pd.read_csv("Segmentation.csv")
OldColumn = list(Segmentation.columns)
Newcolumnw = ['REGION_CENTROID_COL','REGION_CENTROID_ROW','REGION_PIXEL_COUNT',
 'SHORT_LINE_DENSITY_5',
 'SHORT_LINE_DENSITY_2',
 'VEDGE_MEAN',
 'VEDGE_SD',
 'HEDGE_MEAN',
 'HEDGE_SD',
 'INTENSITY_MEAN',
 'RAWRED_MEAN',
 'RAWBLUE_MEAN',
 'RAWGREEN_MEAN',
 'EXRED_MEAN',
 'EXBLUE_MEAN',
 'EXGREEN_MEAN',
 'VALUE_MEAN',
 'SATURATION_MEAN',
 'HUE_MEAN']

SegmentationC.columns = Newcolumnw

## Data Clean Up

In [3]:
def DropDeadData(Data):
    NumberOfDataDropped = 0
    DataColums = list(Data.columns)
    for Attribute in DataColums:
        for i in Data.query(''+Attribute+' == "?"').index:
            Data=Data.drop(i)
            NumberOfDataDropped=NumberOfDataDropped + 1
    print(NumberOfDataDropped)

def replace_Text(Data, Column):
    j = 1 
    for x in Column:
        Data = Data.replace(x, j)
        j = j+1 
    return Data

def shuffle_split_data(X,val=10):
    arr_rand = np.random.rand(len(X))
    split = arr_rand < np.percentile(arr_rand, val)

    X_train = X[split]
    X_test =  X[~split]

    #print len(X_Train), len(X_Test)
    return X_test,X_train

In [4]:
########################################################   
#Avalone Data Clean up
########################################################
conditions = [
    (Abalone['Sex'] == "I"),
    (Abalone['Sex'] == "F"),
    (Abalone['Sex'] == "M")
    ]

# create a list of the values we want to assign for each condition
values = [1, 2, 3]

# create a new column and use np.select to assign values to it using our lists as arguments
Abalone['Sex'] = np.select(conditions, values)


In [5]:
########################################################   
#House Vote Data Clean up
########################################################
conditions = [
    (HouseVote['Class'] == "democrat"),
    (HouseVote['Class'] == "republican")
    ]

# create a list of the values we want to assign for each condition
values = [1, 2]

# create a new column and use np.select to assign values to it using our lists as arguments
HouseVote['Class'] = np.select(conditions, values)

HouseVote = HouseVote.replace('y',1)
HouseVote = HouseVote.replace('n',0)
HouseVote = HouseVote.replace('?',2)

In [6]:
########################################################   
#Machine Data Clean up
########################################################
Vendors = ['amdahl', 'apollo', 'basf', 'bti', 'burroughs', 'c.r.d', 'cdc',
       'cambex', 'dec', 'dg', 'formation', 'four-phase', 'gould', 'hp',
       'harris', 'honeywell', 'ibm', 'ipl', 'magnuson', 'microdata',
       'nas', 'ncr', 'nixdorf', 'perkin-elmer', 'prime', 'siemens',
       'sperry', 'sratus', 'wang']

Models = ['470v/7', '470v/7a', '470v/7b', '470v/7c', '470v/b', '580-5840',
       '580-5850', '580-5860', '580-5880', 'dn320', 'dn420', '7/65',
       '7/68', '5000', '8000', 'b1955', 'b2900', 'b2925', 'b4955',
       'b5900', 'b5920', 'b6900', 'b6925', '68/10-80', 'universe:2203t',
       'universe:68', 'universe:68/05', 'universe:68/137',
       'universe:68/37', 'cyber:170/750', 'cyber:170/760',
       'cyber:170/815', 'cyber:170/825', 'cyber:170/835', 'cyber:170/845',
       'omega:480-i', 'omega:480-ii', 'omega:480-iii', '1636-1',
       '1636-10', '1641-1', '1641-11', '1651-1', 'decsys:10:1091',
       'decsys:20:2060', 'microvax-1', 'vax:11/730', 'vax:11/750',
       'vax:11/780', 'eclipse:c/350', 'eclipse:m/600', 'eclipse:mv/10000',
       'eclipse:mv/4000', 'eclipse:mv/6000', 'eclipse:mv/8000',
       'eclipse:mv/8000-ii', 'f4000/100', 'f4000/200', 'f4000/200ap',
       'f4000/300', 'f4000/300ap', '2000/260', 'concept:32/8705',
       'concept:32/8750', 'concept:32/8780', '3000/30', '3000/40',
       '3000/44', '3000/48', '3000/64', '3000/88', '3000/iii', '100',
       '300', '500', '600', '700', '80', '800', 'dps:6/35', 'dps:6/92',
       'dps:6/96', 'dps:7/35', 'dps:7/45', 'dps:7/55', 'dps:7/65',
       'dps:8/44', 'dps:8/49', 'dps:8/50', 'dps:8/52', 'dps:8/62',
       'dps:8/20', '3033:s', '3033:u', '3081', '3081:d', '3083:b',
       '3083:e', '370/125-2', '370/148', '370/158-3', '38/3', '38/4',
       '38/5', '38/7', '38/8', '4321', '4331-1', '4331-11', '4331-2',
       '4341', '4341-1', '4341-10', '4341-11', '4341-12', '4341-2',
       '4341-9', '4361-4', '4361-5', '4381-1', '4381-2', '8130-a',
       '8130-b', '8140', '4436', '4443', '4445', '4446', '4460', '4480',
       'm80/30', 'm80/31', 'm80/32', 'm80/42', 'm80/43', 'm80/44',
       'seq.ms/3200', 'as/3000', 'as/3000-n', 'as/5000', 'as/5000-e',
       'as/5000-n', 'as/6130', 'as/6150', 'as/6620', 'as/6630', 'as/6650',
       'as/7000', 'as/7000-n', 'as/8040', 'as/8050', 'as/8060',
       'as/9000-dpc', 'as/9000-n', 'as/9040', 'as/9060', 'v8535:ii',
       'v8545:ii', 'v8555:ii', 'v8565:ii', 'v8565:ii-e', 'v8575:ii',
       'v8585:ii', 'v8595:ii', 'v8635', 'v8650', 'v8655', 'v8665',
       'v8670', '8890/30', '8890/50', '8890/70', '3205', '3210', '3230',
       '50-2250', '50-250-ii', '50-550-ii', '50-750-ii', '50-850-ii',
       '7.521', '7.531', '7.536', '7.541', '7.551', '7.561', '7.865-2',
       '7.870-2', '7.872-2', '7.875-2', '7.880-2', '7.881-2',
       '1100/61-h1', '1100/81', '1100/82', '1100/83', '1100/84',
       '1100/93', '1100/94', '80/3', '80/4', '80/5', '80/6', '80/8',
       '90/80-model-3', '32', 'vs-100', 'vs-90']


Machine = replace_Text(Machine, Vendors)   
Machine = replace_Text(Machine, Models)  



In [7]:
########################################################   
#Forest Fire Data Clean up
########################################################
Days = ['mon', 'tue','wed', 'thu','fri', 'sat', 'sun']
Months = ['jan','feb','mar', 'apr', 'may','jun', 'jul','aug', 'sep', 'oct','nov','dec']

ForestFire = replace_Text(ForestFire, Days)
ForestFire = replace_Text(ForestFire, Months)

Split Data 90% for Training and 10% for testing. 

In [8]:
def shuffle_split_data(X,val=10):
    arr_rand = np.random.rand(X.shape[0])
    split = arr_rand < np.percentile(arr_rand, val)

    X_train = X[split]
    X_test =  X[~split]

    #print len(X_Train), len(X_Test)
    return X_test,X_train

In [9]:
glass_x_train, glass_x_test = shuffle_split_data(Glass,10)
votes_x_train, votes_x_test = shuffle_split_data(HouseVote,10)
abalone_x_train, abalone_x_test = shuffle_split_data(Abalone,10)
fires_x_train, fires_x_test = shuffle_split_data(ForestFire,10)
machine_x_train, machine_x_test = shuffle_split_data(Machine,10)
image_x_train, image_x_test = shuffle_split_data(SegmentationC,10)

## K Nearest Neighbor

In [10]:
# Transfer the dataframe to list
AbaloneData = Abalone.values.tolist()
ForestFireData = ForestFire.values.tolist()
GlassData = Glass.values.tolist()
HouseVoteData = HouseVote.values.tolist()
MachineData = Machine.values.tolist()
SegmentationCData = SegmentationC.values.tolist()


In [11]:
# Evaluate an algorithm using a cross validation split
def evaluate_algorithm(dataset, algorithm, n_folds, *args):
    folds = cross_validation_split(dataset, n_folds)
    scores = list()
    for fold in folds:
        train_set = list(folds)
        train_set.remove(fold) #
        train_set = sum(train_set, [])
        test_set = list()
        for row in fold:
            row_copy = list(row)
            test_set.append(row_copy)
            row_copy[-1] = None
        predicted = algorithm(train_set, test_set, *args)
        actual = [row[-1] for row in fold]
        accuracy = accuracy_metric(actual, predicted)
        scores.append(accuracy)
    return scores

In [12]:
# Find the min and max values for each column
def dataset_minmax(dataset):
    minmax = list()
    for i in range(len(dataset[0])):
        col_values = [row[i] for row in dataset]
        value_min = min(col_values)
        value_max = max(col_values)
        minmax.append([value_min, value_max])
    return minmax

# Rescale dataset columns to the range 0-1
def normalize_dataset(dataset, minmax):
    for row in dataset:
        for i in range(len(row)):
            row[i] = (row[i] - minmax[i][0]) / (minmax[i][1] - minmax[i][0])
            
# Calculate accuracy percentage
def accuracy_metric(actual, predicted):
    correct = 0
    for i in range(len(actual)):
        if actual[i] == predicted[i]:
            correct += 1
    return correct / float(len(actual)) * 100.0

In [13]:
# Split a dataset into k folds
def cross_validation_split(dataset, n_folds):
    dataset_split = list()
    dataset_copy = list(dataset)
    fold_size = int(len(dataset) / n_folds)
    for _ in range(n_folds):
        fold = list()
        while len(fold) < fold_size:
            index = randrange(len(dataset_copy))
            fold.append(dataset_copy.pop(index))
        dataset_split.append(fold)
    return dataset_split

In [14]:
# Calculate the Euclidean distance between two vectors
def euclidean_distance(row1, row2):
    distance = 0.0
    for i in range(len(row1)-1):
        distance += (row1[i] - row2[i])**2
    return math.sqrt(distance)

In [79]:
# Locate the most similar neighbors
def get_neighbors(train, test_row, num_neighbors):
    distances = list()
    for train_row in train:
        dist = euclidean_distance(test_row, train_row)
        distances.append((train_row, dist))
    distances.sort(key=lambda tup: tup[1])
    neighbors = list()
    for i in range(num_neighbors):
        neighbors.append(distances[i][0])
#     print('Test Row')
#     print(test_row)
#     print('\n')
#     print('The Neighbors are:') 
#     print(neighbors)
#     time.sleep(2)
    return neighbors
 
# Make a prediction with neighbors
def predict_classification(train, test_row, num_neighbors):
    neighbors = get_neighbors(train, test_row, num_neighbors)
    output_values = [row[-1] for row in neighbors]
    prediction = max(set(output_values), key=output_values.count)
#     print('Prediction is : ')
#     print(prediction)
#     print('\n')
    return prediction
 
# kNN Algorithm
def k_nearest_neighbors(train, test, num_neighbors):
    predictions = list()
    for row in test:
        output = predict_classification(train, row, num_neighbors)
        predictions.append(output)
    return(predictions)

In [16]:
def FindBestK(till, Data):
    Accuracy = 0
    BestK = 0
    for K in range(1,till):
        n_folds = 5
        scores = evaluate_algorithm(Data, k_nearest_neighbors, n_folds, K)
        j = (sum(scores)/float(len(scores)))
        if Accuracy <= j:
            BestK = K
            Accuracy = j
        scores = 0
    return BestK

In [17]:
def WorstK(till, Data):
    Accuracy = 100
    BestK = 0
    for K in range(1,till):
        n_folds = 5
        scores = evaluate_algorithm(Data, k_nearest_neighbors, n_folds, K)
        j = (sum(scores)/float(len(scores)))
        print(j)
        if Accuracy >= j:
            BestK = K
            Accuracy = j
        scores = 0
    return BestK

In [18]:
#The constants have been placed after running this once
# #Store the Best K
# GlassBestK = FindBestK(6, GlassData)
# FireBestK = FindBestK(10, ForestFireData)
# AbaloneBestK = FindBestK(6, AbaloneData)
# VoteBestK = FindBestK(7, HouseVoteData)
# MachineBeskK = FindBestK(7, MachineData)
# SegBestK = FindBestK(10, SegmentationCData)

# #Store the Worst K
# GlassWorstK = WorstK(10,GlassData)
# FireWorstK = WorstK(10,ForestFireData)
# AbaloneWorstK = WorstK(6,AbaloneData)
# VoteWorstK = WorstK(7,HouseVoteData)
# MachineWorstK = WorstK(7,MachineData)
# SegWorstK = WorstK(10,SegmentationCData)

In [19]:
#Constants Created Using the Function Above

GlassBestK = 2
FireBestK = 9
AbaloneBestK = 5
VoteBestK = 4
MachineBeskK = 1
SegBestK = 4

GlassWorstK = 8
FireWorstK = 1
AbaloneWorstK = 1
VoteWorstK = 2
MachineWorstK = 6
SegWorstK = 6

# ENNTOP(HouseVote, 5, 7)

In [20]:
AbaloneEditedData = Abalone.sort_values(by=['Rings'])

In [21]:
# ENNTOP(AbaloneEditedData, 5, 1)

In [78]:
seed(999)
n_folds = 5
num_neighbors = GlassBestK
start = time.time()
scores = evaluate_algorithm(GlassData, k_nearest_neighbors, n_folds, num_neighbors)
end = time.time()
print('Scores: %s' % scores)
print('Mean Accuracy: %.3f%%' % (sum(scores)/float(len(scores))))
RegularGlassTime = end - start

Test Row
[202.0, 1.51653, 11.95, 0.0, 1.19, 75.18, 2.7, 8.93, 0.0, 0.0, None]


The Neighbors are:
[[203.0, 1.51514, 14.85, 0.0, 2.42, 73.72, 0.0, 8.39, 0.56, 0.0, 7.0], [201.0, 1.51508, 15.15, 0.0, 2.25, 73.5, 0.0, 8.34, 0.63, 0.0, 7.0]]
Prediction is : 
7.0


Test Row
[175.0, 1.5205799999999998, 12.85, 1.61, 2.17, 72.18, 0.76, 9.7, 0.24, 0.51, None]


The Neighbors are:
[[178.0, 1.51937, 13.79, 2.41, 1.19, 72.76, 0.0, 9.77, 0.0, 0.0, 6.0], [179.0, 1.51829, 14.46, 2.24, 1.62, 72.38, 0.0, 9.26, 0.0, 0.0, 6.0]]


KeyboardInterrupt: 

In [23]:
seed(999)
n_folds = 5
num_neighbors = AbaloneBestK
start = time.time()
scores = evaluate_algorithm(AbaloneData, k_nearest_neighbors, n_folds, num_neighbors)
end = time.time()
print('Scores: %s' % scores)
print('Mean Accuracy: %.3f%%' % (sum(scores)/float(len(scores))))
RegularAbaloneTime = end - start

Scores: [23.832335329341316, 21.55688622754491, 22.395209580838323, 23.353293413173652, 28.023952095808387]
Mean Accuracy: 23.832%


In [24]:
seed(900)
n_folds = 5
num_neighbors = FireBestK
start = time.time()
scores = evaluate_algorithm(ForestFireData, k_nearest_neighbors, n_folds, num_neighbors)
end = time.time()
print('Scores: %s' % scores)
print('Mean Accuracy: %.3f%%' % (sum(scores)/float(len(scores))))

RegularFireTime = end - start

Scores: [52.42718446601942, 55.33980582524271, 38.83495145631068, 43.689320388349515, 42.71844660194174]
Mean Accuracy: 46.602%


In [25]:
n_folds = 5
num_neighbors = VoteBestK
start = time.time()
scores = evaluate_algorithm(HouseVoteData, k_nearest_neighbors, n_folds, num_neighbors)
end = time.time()
print('Scores: %s' % scores)
print('Mean Accuracy: %.3f%%' % (sum(scores)/float(len(scores))))

RegularVoteTime = end - start

Scores: [93.02325581395348, 94.18604651162791, 90.69767441860465, 91.86046511627907, 91.86046511627907]
Mean Accuracy: 92.326%


In [26]:
n_folds = 5
num_neighbors = MachineBeskK
start = time.time()
scores = evaluate_algorithm(MachineData, k_nearest_neighbors, n_folds, num_neighbors)
end = time.time()
print('Scores: %s' % scores)
print('Mean Accuracy: %.3f%%' % (sum(scores)/float(len(scores))))

RegularMachineTime = end - start

Scores: [41.46341463414634, 60.97560975609756, 34.146341463414636, 46.34146341463415, 56.09756097560976]
Mean Accuracy: 47.805%


In [27]:
n_folds = 5
num_neighbors = SegBestK
start = time.time()
scores = evaluate_algorithm(SegmentationCData, k_nearest_neighbors, n_folds, num_neighbors)
end = time.time()
print('Scores: %s' % scores)
print('Mean Accuracy: %.3f%%' % (sum(scores)/float(len(scores))))

RegularSegTime = end - start

Scores: [2.4390243902439024, 0.0, 0.0, 7.317073170731707, 7.317073170731707]
Mean Accuracy: 3.415%


### Edited Nearest Neighbor

In [61]:
# Make a classification prediction with neighbors
def predict_classification_edited(train, test_row, num_neighbors):
    neighbors = get_neighbors(train, test_row, num_neighbors)
    output_values = [row[-1] for row in neighbors]
    prediction = max(set(output_values), key=output_values.count)
    return prediction

def EditedKNN(PDDataset, K):
    J = 0
    K = 0
    DoneDropping = True
    while(DoneDropping):
        K = K+1
        J = 0
        DataList = PDDataset.values.tolist()
        PrviousJ = len(DataList)
#         print('Perevios J : '+ str(PrviousJ))
        for x in range(len(DataList)):
            prediction = predict_classification_edited(DataList, DataList[x], K)
            if DataList[x][-1] != prediction:
                PDDataset = PDDataset.drop(index = x)
                J = J + 1
        print(J)
        if J < 1:
            DoneDropping = False    
        PDDataset = PDDataset.reset_index(drop=True)
                
    return PDDataset.values.tolist()


def ENNTOP(DataSet, Bestk, WorstK):
    EditedKNNData = EditedKNN(DataSet, WorstK)
    n_folds = 5
    num_neighbors = Bestk
    start = time.time()
    scores = evaluate_algorithm(EditedKNNData, k_nearest_neighbors, n_folds, num_neighbors)
    end = time.time()
    print('Scores: %s' % scores)
    print('Mean Accuracy: %.3f%%' % (sum(scores)/float(len(scores))))
    Time = end - start
    print(Time)

In [80]:
ForestFire.describe()

,X_Cord,Y_Cord,Month,Day,FFMC,DMC,DC,ISI,Temp,RH,Wind,Rain,Area
count,517.000000,517.000000,517.000000,517.000000,517.000000,517.000000,517.000000,517.000000,517.000000,517.000000,517.000000,517.000000,517.000000
mean,4.669246,4.299807,7.475822,4.259188,90.644681,110.872340,547.940039,9.021663,18.889168,44.288201,4.017602,0.021663,12.847292
std,2.313778,1.229900,2.275990,2.072929,5.520111,64.046482,248.066192,4.559477,5.806625,16.317469,1.791653,0.295959,63.655818
min,1.000000,2.000000,1.000000,1.000000,18.700000,1.100000,7.900000,0.000000,2.200000,15.000000,0.400000,0.000000,0.000000
25%,3.000000,4.000000,7.000000,2.000000,90.200000,68.600000,437.700000,6.500000,15.500000,33.000000,2.700000,0.000000,0.000000
50%,4.000000,4.000000,8.000000,5.000000,91.600000,108.300000,664.200000,8.400000,19.300000,42.000000,4.000000,0.000000,0.520000
75%,7.000000,5.000000,9.000000,6.000000,92.900000,142.400000,713.900000,10.800000,22.800000,53.000000,4.900000,0.000000,6.570000
max,9.000000,9.000000,12.000000,7.000000,96.200000,291.300000,860.600000,56.100000,33.300000,100.000000,9.400000,6.400000,1090.840000


In [81]:
# This is to Show ENN
MachineEditedKNN = EditedKNN(ForestFire, 7)
len(MachineEditedKNN)

9
159
68
25
10
1
0


245

In [29]:
print(RegularAbaloneTime)
print(RegularFireTime)
print(RegularVoteTime)
print(RegularMachineTime)
print(RegularSegTime)

101.102046251297
1.715505838394165
1.7842471599578857
0.28403520584106445
0.37105655670166016


In [62]:
MachineEditedKNN = EditedKNN(Machine, 7)

n_folds = 5
num_neighbors = 1
scores = evaluate_algorithm(MachineEditedKNN, k_nearest_neighbors, n_folds, num_neighbors)
print('Scores: %s' % scores)
print('Mean Accuracy: %.3f%%' % (sum(scores)/float(len(scores))))

0
Scores: [34.146341463414636, 56.09756097560976, 48.78048780487805, 43.90243902439025, 43.90243902439025]
Mean Accuracy: 45.366%


In [31]:
AbaloneDataEditedKNN = EditedKNN(Abalone, 7)

n_folds = 5
num_neighbors = 5
scores = evaluate_algorithm(AbaloneDataEditedKNN, k_nearest_neighbors, n_folds, num_neighbors)
print('Scores: %s' % scores)
print('Mean Accuracy: %.3f%%' % (sum(scores)/float(len(scores))))

Scores: [23.233532934131738, 20.838323353293415, 22.51497005988024, 24.311377245508982, 24.910179640718564]
Mean Accuracy: 23.162%


In [32]:
ForestFireEditedKNN = EditedKNN(ForestFire, 7)

n_folds = 5
num_neighbors = 9
scores = evaluate_algorithm(ForestFireEditedKNN, k_nearest_neighbors, n_folds, num_neighbors)
print('Scores: %s' % scores)
print('Mean Accuracy: %.3f%%' % (sum(scores)/float(len(scores))))

Scores: [100.0, 100.0, 100.0, 100.0, 100.0]
Mean Accuracy: 100.000%


In [33]:
GlassEditedKNN = EditedKNN(Glass, 7)

n_folds = 5
num_neighbors = 5
start = time.time()
scores = evaluate_algorithm(GlassEditedKNN, k_nearest_neighbors, n_folds, num_neighbors)
end = time.time()
print('Scores: %s' % scores)
print('Mean Accuracy: %.3f%%' % (sum(scores)/float(len(scores))))
EditedGlassKNNTime = end - start
print(EditedGlassKNNTime)
print(RegularGlassTime)


Scores: [95.23809523809523, 97.61904761904762, 100.0, 95.23809523809523, 100.0]
Mean Accuracy: 97.619%
0.25660133361816406
0.47237563133239746


In [34]:
HouseVoteEditedKNN = EditedKNN(HouseVote, 7)

n_folds = 5
num_neighbors = 6
scores = evaluate_algorithm(HouseVoteEditedKNN, k_nearest_neighbors, n_folds, num_neighbors)
print('Scores: %s' % scores)
print('Mean Accuracy: %.3f%%' % (sum(scores)/float(len(scores))))

Scores: [91.86046511627907, 94.18604651162791, 90.69767441860465, 93.02325581395348, 91.86046511627907]
Mean Accuracy: 92.326%


In [36]:
SegmentationCEditedKNN = EditedKNN(SegmentationC, 7)

n_folds = 5
num_neighbors = 6
scores = evaluate_algorithm(SegmentationCEditedKNN, k_nearest_neighbors, n_folds, num_neighbors)
print('Scores: %s' % scores)
print('Mean Accuracy: %.3f%%' % (sum(scores)/float(len(scores))))

Scores: [0.0, 2.4390243902439024, 2.4390243902439024, 0.0, 2.4390243902439024]
Mean Accuracy: 1.463%


### Condensed Nearest Neighbor

In [82]:
def CondensedKNN(PDDataset, K):
    CopyDataSet = []
    DataList = PDDataset.values.tolist()
    for x in range(len(DataList)-7):
        prediction = predict_classification_edited(DataList, DataList[x], K)
        if DataList[x][-1] == prediction:
            CopyDataSet.append(x)
            #print(CopyDataSet)
            #print(PDDataset.iloc[[x]])
            #time.sleep(0.1)
            #print(CopyDataSet)
            #PDDataset = PDDataset.drop(index = x)
    print(CopyDataSet)
    PDDataset = PDDataset.iloc[CopyDataSet]
    PDDataset = PDDataset.reset_index(drop=True)
    return PDDataset.values.tolist()

# def EditedCNN(PDDataset, K):
#     J = 0
#     K = 0
#     CopyDataSet = []
#     DoneDropping = True
#     while(DoneDropping):
#         K = K+1
#         J = 0
#         DataList = PDDataset.values.tolist()
#         PrviousJ = len(DataList)
# #         print('Perevios J : '+ str(PrviousJ))
#         for x in range(len(DataList)):
#             prediction = predict_classification_edited(DataList, DataList[x], K)
#             if DataList[x][-1] == prediction:
#                 CopyDataSet.append(x)
#                 #PDDataset = PDDataset.drop(index = x)
#                 J = J + 1
# #         print(J)
#         if J < 1 or K > 1:
#             DoneDropping = False    
#         print(PDDataset.shape)
#         PDDataset = PDDataset.iloc[CopyDataSet]
#         PDDataset = PDDataset.reset_index(drop=True)
#     print(CopyDataSet)          
#     return PDDataset.values.tolist()

def CNN(DataSet,Bestk,WorsK):
    CondensedData = CondensedKNN(DataSet, 2)
    n_folds = 5
    num_neighbors = 5
    start = time.time()
    scores = evaluate_algorithm(CondensedData, k_nearest_neighbors, n_folds, num_neighbors)
    end = time.time()
    print('Scores: %s' % scores)
    print('Mean Accuracy: %.3f%%' % (sum(scores)/float(len(scores))))
    Time = end - start
    print(Time)

In [68]:
CNN(Machine, 5, 5)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 11, 15, 16, 18, 19, 20, 22, 23, 25, 26, 28, 29, 30, 31, 32, 33, 34, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 52, 53, 55, 56, 57, 58, 59, 60, 64, 66, 69, 70, 71, 72, 73, 74, 76, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 137, 139, 140, 141, 142, 143, 145, 146, 147, 148, 150, 151, 154, 155, 157, 158, 161, 164, 165, 172, 173, 174, 175, 176, 177, 178, 179, 180, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 193, 194, 195, 196, 197, 198, 199, 200]
Scores: [37.5, 28.125, 31.25, 31.25, 31.25]
Mean Accuracy: 31.875%
0.22815799713134766


In [39]:
CNN(HouseVote, 5, 7)

Scores: [91.46341463414635, 89.02439024390245, 97.5609756097561, 97.5609756097561, 96.34146341463415]
Mean Accuracy: 94.390%
1.5992155075073242


In [40]:
CNN(Abalone, 5, 2)

Scores: [35.24752475247524, 36.43564356435643, 36.43564356435643, 36.633663366336634, 34.05940594059406]
Mean Accuracy: 35.762%
34.19151735305786


In [41]:
CNN(Glass, 5, 5)

Scores: [100.0, 100.0, 90.2439024390244, 97.5609756097561, 100.0]
Mean Accuracy: 97.561%
0.22760272026062012


In [43]:
CNN(SegmentationC, 5, 5)

Scores: [9.090909090909092, 0.0, 18.181818181818183, 0.0, 9.090909090909092]
Mean Accuracy: 7.273%
0.02652740478515625


In [44]:
CNN(ForestFire, 5, 5)

Scores: [72.46376811594203, 68.11594202898551, 68.11594202898551, 68.11594202898551, 63.76811594202898]
Mean Accuracy: 68.116%
0.7428293228149414
